<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/HPC/BarraQuelca_GuidoAlberto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se realizara la **Adición Matricial** A y B, el resultado de la Adición se guardara en la matriz C.
Hay varias definiciones de Adición Matricial, por mencionar algunos:

**Suma:** denotada por el signo +, suma el componente de la matriz A y el componente de la matriz B y el resultado de esta suma es un componente de la matriz C. Realiza la suma componente a componente de las matrices.

**Suma Directa**: denotada por ⊕, no suma componente a componente como la Suma, sino que ambas matrices conviven en una matriz C de mayor dimensión que la matrices A Y B. Cada componente de la diagonal principal de la matriz C es una matriz, el resto de los componentes tiene ceros. Esta definición de Adición no necesita que sus matrices tengan las mismas dimensiones.

En este ejemplo se realizará la Adición Matricial de **Suma**, la cual es la más conocida y más utilizada. Como se explicó antes la Adición Matricial de **Suma** realiza la suma componente a componente de las matrices, además de esto las matrices deben tener las mismas dimensiones, si las dimensiones no son igual no se puede realizar la **Suma**

El desarrollo teórico de la **Suma** de matrices se mostrará a continuación. Realiza la suma componente a componente de las matrices

\begin{aligned}\mathbf {A} +\mathbf {B} &={\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{m1}&a_{m2}&\cdots &a_{mn}\\\end{bmatrix}}+{\begin{bmatrix}b_{11}&b_{12}&\cdots &b_{1n}\\b_{21}&b_{22}&\cdots &b_{2n}\\\vdots &\vdots &\ddots &\vdots \\b_{m1}&b_{m2}&\cdots &b_{mn}\\\end{bmatrix}}\\&={\begin{bmatrix}a_{11}+b_{11}&a_{12}+b_{12}&\cdots &a_{1n}+b_{1n}\\a_{21}+b_{21}&a_{22}+b_{22}&\cdots &a_{2n}+b_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{m1}+b_{m1}&a_{m2}+b_{m2}&\cdots &a_{mn}+b_{mn}\\\end{bmatrix}}\\\end{aligned}


La condición para poder realizar la suma de matrices es que la cantidad de filas de la matriz A tiene que ser igual a la cantidad de filas de la matriz B, y además cantidad de columnas de la matriz A tiene que ser igual a la cantidad de columnas de la matriz B, en la imagen de arriba se ve como la matriz A y B cumple con esta condición.

EL objetivo es enseñar el funcionamiento del Lenguaje Python, C++, OpenMP y el manejo de la operación de matriz a bajo nivel. El ejemplo es ilustrativo para entender y utilizar los Core que tiene un procesador, realizar proceso en paralelo y sacar conclusiones de la biblioteca OpenMP programar un código que se va a realizar de forma paralela en los Core del procesador

# 2 Armado del ambiente
Se crea un archivo .cpp con el nombre de suma_matriz_axpy, este archivo contiene el codigo él cual ejecutará la suma de matrices de forma secuencial y tambien la suma de matrices de forma paralela utilizando los Core que tiene el procesador.

In [4]:
# Codigo C++.
code = """
// Axpy con OpenMP, usando C++, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  TIEMPO_INI( HTH_TOTAL )

  // validar parametros.
  if( argc != 4 )
  {
    std::cout<<argv[1]<<std::endl;
    std::cout<<argv[2]<<std::endl;
    std::cout<<argv[3]<<std::endl;
    std::cerr<< " Error en los parametros, los cantidad de parametros deben ser 3:"<<std::endl<<" (alfa), (beta), (Tamaño de la matriz cantidad_N)."<<argc<<std::endl;
    exit( -1 );
  }

  // Obtener parametros
  float alfa     = atof( argv[1] );
  float beta     = atoi( argv[2] );
  int cantidad_N = atoi( argv[3] );
  
  // Defino la memoria de las Matriz A, B y C. Inicializo con unos la matriz A y con el numero dos la matriz B
  std::vector<std::vector<double>> matriz_A(cantidad_N, std::vector<double> (cantidad_N, 1));
  std::vector<std::vector<double>> matriz_B(cantidad_N, std::vector<double> (cantidad_N, 2));
  std::vector<std::vector<double>> matriz_C_secuencial(cantidad_N, std::vector<double> (cantidad_N, 0));
  std::vector<std::vector<double>> matriz_C_paralela(cantidad_N, std::vector<double> (cantidad_N, 0));

  // Realizo la función Axpy en forma secuencial.

  TIEMPO_INI( HTH_AXPY_SEC )

  for (int i=0;i<cantidad_N;i++)
  {
    for (int j=0;j<cantidad_N;j++)
    {
      matriz_C_secuencial[i][j] = alfa*matriz_A[i][j] + beta*matriz_B[i][j];
    }
  }

  TIEMPO_FIN( HTH_AXPY_SEC )


  // Realizo la función Axpy con OpenMP, paralela.

  TIEMPO_INI( HTH_AXPY_OMP )

  for (int i=0;i<cantidad_N;i++)
  {  
    #pragma omp parallel for  
    for (int j=0;j<cantidad_N;j++)
    {
      matriz_C_paralela[i][j] = alfa*matriz_A[i][j] + beta*matriz_B[i][j];
    }
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Muestro los resultados.
  /*
  std::cout<<" matriz paralela :"<<std::endl;
  std::cout<<"["; 
  for(int i=0;i<cantidad_N;i++)
  {
    for(int c=0;c<cantidad_N;c++)
    {
      std::cout<<matriz_C_paralela[i][c]<< ", ";
    }
    std::cout<<std::endl;
  }
  std::cout<<"]"<<std::endl<<std::endl; 

  std::cout<<" matriz secuencial :"<<std::endl;
  std::cout<<"["; 
  for(int i=0;i<cantidad_N;i++)
  {
    for(int c=0;c<cantidad_N;c++)
    {
      std::cout<<matriz_C_secuencial[i][c]<< ", ";
    }
    std::cout<<std::endl;
  }
  std::cout<<"]"<<std::endl; 
  */

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"EJERCICIO DE SUMA DE MATRICES"<<std::endl;
 std::cout<<"C = alfa*A + beta*B"<<std::endl;
 std::cout<<"se realizo "<<cantidad_N*cantidad_N<<" operaciones de suma"<<std::endl;
 std::cout<<"Valor ALFA       : "<<alfa<<std::endl;
 std::cout<<"Valor BETA       : "<<beta<<std::endl;
 std::cout<<"MATRIZ CUADRADA  : "<<cantidad_N<<std::endl; 
 std::cout<<"Valores Reales   : "<<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / 2;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

"""
text_file = open("suma_matriz_axpy.cpp", "w")
text_file.write(code)
text_file.close()

## 2.1 Compila el codigo de C++.

In [6]:
!g++ -o suma_matriz -fopenmp suma_matriz_axpy.cpp

# 3 Desarrollo
Ejecución del programa que realiza la suma de matrices


In [13]:
try:
  %env OMP_NUM_THREADS=2
  !./suma_matriz 2 3 18000
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

env: OMP_NUM_THREADS=2
EJERCICIO DE SUMA DE MATRICES
C = alfa*A + beta*B
se realizo 324000000 operaciones de suma
Valor ALFA       : 2
Valor BETA       : 3
MATRIZ CUADRADA  : 18000
Valores Reales   : 
Tiempo TOTAL     : 18421.6 [ms]
Tiempo axpy Sec  : 5836.09 [ms]
Tiempo axpy Omp  : 4983.58 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 5836.09 / 4983.58 = 1.17106
Eficiencia       : SpeedUp/nro procesadores            : 1.17106 / 2 = 0.585532
Coste Sec        : nro procesadores*Tiempo             : 1 * 5836.09 = 5836.09
Coste Omp        : nro procesadores*Tiempo             : 2 * 4983.58 = 9967.16
Funcion Overhead : Coste Omp - tiempo Secuencial       : 9967.16 - 5836.09 = 4131.07

Valores Ideal: 
Tiempo axpy Sec  : 5836.09 [ms]
Tiempo axpy Omp  : 2918.04 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 5836.09 / 2918.04 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo             : 1 * 5836.0

---
# 4 Tabla de pasos


 Procesador | lengueaje   | Funciòn 					   | Detalle
------------|----------   |--------------------------------|----------------
CPU         | Python      |  open()                        | Abrir un archivo para escritura
CPU         | Python      |  write()                       | Escribir el archivo
CPU         | Python      |  close()                       | Cerrar el archivo
CPU   	    | Python      |  !g++ -o obj -fopenmp c.cpp    | Compilar el archivo .cpp, y crear ejectable
CPU         | Python      |  %env OMP_NUM_THREADS=n        | Cantidad de threads n
CPU         | Python      |  !./suma_matriz 2 5 18000      | Ejecutar el programa openMP
CPU         | C++         |  TIEMPO_INI()                  | Inicio del tiempo
CPU         | C++         |  TIEMPO_FIN()                  | Finaliza el tiempo
CPU         | C++         |  TIEMPO_GET()                  | Obtener el tiempo guardado
CPU         | C++, openMP |  omp_get_num_procs()           | Obtener la cantidad de procesadores
CPU         | C++, openMP |  omp_get_thread_num()          | Obtener numero del hilo
CPU		    | C++, openMP |  #pragma omp parallel for      | Crear los hilos en base a la sentencia for


#5 Conclusiones

Se realizaron varias pruebas con matrices de distintos tamaños. Colab nos brinda un procesador y una memoria fijas, el cual nos limita al momento de realizar las pruebas, un ejemplo de esto es que no se puede probar matrices mayores a 18000x18000, debido a que se llena la menoria que nos ofrece Colab.

**Pruebas con Cantidad_N<1000:** En estas pruebas se observó que el tiempo de ejecución para la suma de matrices son aproximadamente igual para Cantidad_N = 1000, tanto para la forma secuencial, como para la forma paralela que utiliza los Core del procesador. Al disminuir la Cantidad_N el tiempo de ejecución de la forma secuencial es mejor que el tiempo de ejecución de la forma paralela esto es debido a que openMP utiliza hilos, la creación de los hilos y su cambio de contexto hacen que el tiempo que ejecución del algoritmo aumente.

**Pruebas con Cantidad_N>1000:** En estas pruebas se observo que el tiempo de ejecución para la suma de matrices son distintos, al utilizar los Core del procesador se observa que hay una mejora en la eficiencia y el tiempo es menor, en comparación con la forma secuencial.

**Pruebas variando la cantidad de hilos:** En esta prueba se observo un baja significativa de la performance del 30% o mayor, en comparación con la forma secuencial, al aumentar la cantidad de hilos el tiempo de ejecución de la forma paralela aumenta, esto es debido a que el tiempo de creación de los hilos y su cambio de contexto impacta en el tiempo total, si bien openMP mejora y reduce el tiempo de creacion y cambio de contexto de los hilos, al tener varios hilos el tiempo aumenta.

Se concluye que la cantidad de elementos, los hilos y los Core del procesador son factores que influyen en el tiempo de ejecución del algoritmo. Estos tres factores son importantes y hay que tenerlos en cuenta debido a que el tiempo de ejecución utilizando openMP es peor que la ejecución secuencial si se tiene un conjunto pequeño de elementos a procesar o si se tiene muchos hilos con un procesador que tiene pocos Core. Para un conjunto grande de elementos y una cantidad de hilos acorde a la cantidad de Core del procesador el tiempo que ejecución del algoritmo utilizando openMP es mejor que el tiempo de ejecución secuencial.

---
# 6 Bibliografía

[1] Suma de matrices, conceptual: [Pagina economipedia](https://economipedia.com/definiciones/suma-de-matrices.html)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Adicíon Matricial: [WIKI](https://es.wikipedia.org/wiki/Adici%C3%B3n_matricial)

[4] OpenMP: [PDF](http://so-unlam.com.ar/material-clase/HPC/openmp-4.5.pdf)

[5] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)



